# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [14]:
from xml.etree import ElementTree as ET
import numpy as np
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [15]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [16]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [17]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [18]:
document = ET.parse( './data/mondial_database.xml' )
root =document.getroot()

## Q1 Response: top 10 countries by infrant mortality rates

In [19]:
#Create separate lists for country and infant mortality rates
c= []
i= []

#Iterate through country tag, find country names and corresponding infant mortality indicators.
for element in root.iter('country'):
    if element.find('name') !=None:
        if element.find('infant_mortality') !=None:
            c.append(element.find('name').text) #saves country names into  'c' list
            i.append(float(element.find('infant_mortality').text)) #saves mortality rates into 'i' list
        else:
            continue
                     
    else:
        continue
        
#Create a pandas dataframe from a dictionary containing the list of countries and infant mortality rates
#sort the contents of the dataframe by values (lowest first), show the top 10. Show the dataframe results.
df0= pd.DataFrame({'Country': c, 'Infant Mortality': i}).sort_values(by='Infant Mortality').head(10)

df0 

,Country,Infant Mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


## Q2 Response: Top 10 cities by population

In [25]:
#Lists for population and city

citypop=[]

#Iterate through each country/city. extract city name and the latest population numbers
for elem in root.iterfind('country'):
    for subelem in elem.getiterator('city'):
        if subelem.find('population') != None:
            dict ={}
            dict['city']=subelem.find('name').text
            dict['population']=float(subelem.findall('population')[-1].text)
            citypop.append(dict)
        else:
            continue
           
#Store results in pandas dataframe.Sort values in descending order.Show top 10 values.
df = pd.DataFrame(citypop).sort_values('population', ascending=False).head(10)    
df 

,city,population
1251,Shanghai,22315474.0
707,Istanbul,13710512.0
1421,Mumbai,12442373.0
443,Moskva,11979529.0
1250,Beijing,11716620.0
2594,São Paulo,11152344.0
1252,Tianjin,11090314.0
974,Guangzhou,11071424.0
1467,Delhi,11034555.0
977,Shenzhen,10358381.0


## #Q3 Response: Top 10 ethnic groups

In [87]:

eth_list =[]

#Iterate through country tag. Collect the latest country population figures and ethnicity percentage values.
#Save values in 'eth_list' list.

for element in root.iterfind('country'):
    dict = {} #initialize a dictionary to store values
    
    if element.find('population') != None: #check for value
        for ethnic in element.findall('ethnicgroup'):    
            dict['country']=element.find('name').text
            dict['population'] = int(element.findall('population')[-1].text)
            dict['ethnicgroup'] = ethnic.text
            dict['percentage']=float(ethnic.attrib['percentage'])
            
            eth_list.append(dict)
            
            #clear dict values for next country iteration
            dict = {}
# Store values in pandas format.Calculate percentage of ethnicities.             
df0= pd.DataFrame(eth_list)          
df0['population_group'] = df0.population*df0.percentage/100.0
print(df0.head(5))

#Top 10 ethnic groups.
df0=pd.DataFrame(df0.groupby('ethnicgroup').population_group.sum().sort_values(ascending = False).head(10))
df0


     country ethnicgroup  percentage  population  population_group
0    Albania    Albanian        95.0     2800138      2.660131e+06
1    Albania       Greek         3.0     2800138      8.400414e+04
2     Greece       Greek        93.0    10816286      1.005915e+07
3  Macedonia  Macedonian        64.2     2059794      1.322388e+06
4  Macedonia    Albanian        25.2     2059794      5.190681e+05


,population_group
ethnicgroup,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


## Q4A: Country With the Longest River

In [128]:

rivers = []
#Extract river name, country and length information for each instance of river

for element in root.iterfind('river'):
    dict = {} #initialize a dictionary to store values
    if element.find('length') != None: # check for value
        dict['country'] = element.attrib['country']
        dict['name'] = element.find('name').text
        dict['length'] = float(element.find('length').text)
           
        rivers.append(dict)
        #clear dict values for next river iteration
        dict = {}
        
Theriver =pd.DataFrame(rivers).dropna()
Theriver.sort_values(by = 'length', ascending = False).head(1)

,country,length,name
174,CO BR PE,6448.0,Amazonas


## Q4B: Country with the Largest Lake

In [115]:
lakes = []
#Extract lake name, country and area information for each instance of the lake

for element in root.iterfind('lake'):
    dict = {} #initialize a dictionary to store values
    if element.find('area') != None: # check for value
        dict['country'] = element.attrib['country']
        dict['name'] = element.find('name').text
        dict['area'] = float(element.find('area').text)
           
        lakes.append(dict)
        #clear dict values for next lake iteration
        dict = {}
        
#Clean rows with missing data.Sort values and select the top row.
TheLake =pd.DataFrame(lakes).dropna()
TheLake.sort_values(by = 'area', ascending = False).head(1)

,area,country,name
54,386400.0,R AZ KAZ IR TM,Caspian Sea


## Q4C: Country with Airport at Highest Elevation

In [127]:
airports = []
#Extract lake name, country and elevation for each instance of the airport

for element in root.iterfind('airport'):
    dict = {} #initialize a dictionary to store values
    dict['country'] = element.attrib['country']
    dict['name'] = element.find('name').text

    if element.find('elevation') != None: # check for value
        dict['elevation'] = element.find('elevation').text
                
        airports.append(dict)
        #clear dict values for next airport iteration
        dict = {}
        
        
#Clean rows with missing data.Sort values and select the top row.
TheAirport =pd.DataFrame(airports).dropna()

TheAirport.elevation = TheAirport.elevation.map(float)

TheAirport.sort_values(by = 'elevation', ascending = False).head(1)

,country,elevation,name
80,BOL,4063.0,El Alto Intl
